## Spark에서의 WordCount 

In [ ]:
# pip install pyspark==3.5.1
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("streaming").master("local[*]").getOrCreate()
spark

In [ ]:
spark_df = spark.read.format("text").load('hdfs://localhost:9000/Test_Dir/worddata.txt')
spark_df.printSchema()
spark_df.show()


In [ ]:
from pyspark.sql.functions import split, explode
# 빈칸 기준 단어분리하고 리스트로 저장
# 단어라스트를 Explode(리스트를 각 row로 분리)
words_df = spark_df.select(explode(split(spark_df.value," ")).alias("word"))
words_df.show()

In [ ]:
from pyspark.sql.functions import count
wordcount_df = words_df.groupBy("word").count()
wordcount_df.show()

### Apache Spark Streaming
- Nmap 설치: https://nmap.org  (ncat)
- CMD > ncat -l any_port
- CMD > ncat -C localhost choosen_port

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, explode, count

In [ ]:
spark = (
    SparkSession
    .builder
    .appName("streaming")
    .config("spark.streaming.stopGracefullyOnShutdown",True)
    .master("local[*]")
    .getOrCreate()
)
spark

In [ ]:
spark_df = spark.readStream.format("socket").option("host","localhost").option("port","8080").load()

In [ ]:
words_df  = spark_df.select(explode(split(spark_df.value," ")).alias("word"))
wordcount_df = words_df.groupBy("word").count()

In [ ]:
sq = wordcount_df.writeStream.format("console").outputMode("complete").start()
#sq = agg_df.writeStream.format("console").outputMode("update").start()
sq.awaitTermination(10)
# Return whether the query has terminated or not within 10 seconds

In [ ]:
sq.stop()